# Pre-requisites

In [ ]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

In [ ]:
import os
os.environ['AWS_PROFILE'] = ''

In [ ]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

In [ ]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [ ]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

In [ ]:
# configuration variables
suffix = f"{region}-{account_id}"
stack_name = "Agent"
agent_name = f"{stack_name}-pathology-assistant"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}-{stack_name}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}_{stack_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for runnign Pathology Questions"
agent_instruction = "You are an Pathology Assistant, helping pathologists retrieve information from past pathology reports"
agent_action_group_name = "PathologyActionGroup"
agent_action_group_description = "Actions for Retrieving information from Pathology Reports or from Machine Learning Models"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f"{stack_name}-LambdaExecutionRole"
lambda_function_name = f"{stack_name}-{account_id}-Lambda"
lambda_function_arn = lambda_client.get_function(FunctionName=lambda_function_name)['Configuration']['FunctionArn']

In [ ]:
lambda_function_arn

# Create Lambda Function 

In [ ]:
lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)
lambda_iam_role

# Create Agent

In [ ]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response


In [ ]:
agent_id = response['agent']['agentId']
agent_id

# Agent Action Group

In [ ]:
agent_functions = [
    {
        'name': 'retrieve_existing_pathology_report',
        'description': 'retrieves the pathology report for a given patient id',
        'parameters': {
            "patient_id": {
                "description": "the patient_id of the patient for which we want a pathology report",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'wsi_feature_extraction',
        'description': 'Starts an AWS Batch Job to extracts the features required to run msi classification for a given patient id. Returns the AWS Batch Job Id',
        'parameters': {
            "patient_id": {
                "description": "the patient_id of the patient for which we want to find a WSI image and extract the features ()",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'retrieve_msi_status',
        'description': 'Starts an AWS Batch Job that uses the extracted features of a WSI Image of a given patient_jd to predict microsatellite instability status. Returns an AWS Batch Job Id',
        'parameters': {
            "patient_id": {
                "description": "the patient_id of the patient for which we want the microsatellite instability status",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'check_on_aws_batch_job_status',
        'description': 'Utility function that checks on the Job Status of an AWS Batch Job using the jobId',
        'parameters': {
            "jobId": {
                "description": "the AWS Batch JobId",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'check_on_executed_ml_models',
        'description': 'Utility function that checks if the ml models have been executed for a given patient_id and returns the result',
        'parameters': {
            "patient_id": {
                "description": "the patient_id of the patient for which we want to check the ml models",
                "required": True,
                "type": "string"
            }
        }
    },

]

In [ ]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [ ]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

In [ ]:
lambda_function_name

# Preparing Agent

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

In [ ]:
s3_client = boto3.client('s3')
patient_id='TCGA-3L'

In [ ]:
import boto3
from botocore.client import Config
import json

# Bedrock configuration
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
region = 'us-east-1'
bucket_name = 'sagemaker-us-east-1-343218212359' 
sagemaker_runtime = boto3.client('runtime.sagemaker')
bedrock_agent_client = boto3.client("bedrock-agent-runtime", region_name=region, config=bedrock_config,)
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
ENDPOINT_NAME = "MSI-CLASSIFIER"

In [ ]:
prefix = f"WSI/{patient_id}-"
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
response